In [82]:
import numpy as np
import pandas as pd

In [83]:
def get_namespace(column_value):
    column_value_list = column_value.split(":")
    return column_value_list[0]

In [84]:
def count_checksums(checksum_column):
    if checksum_column == "not_set":
        return 0
    else:
        checksum_list = checksum_column.split(",")
        return len(checksum_list)

In [85]:
def get_members(df, df_collection_members, collection, b_include_pages):
    is_collection_member =  df['isMemberOfCollection'] == collection
    member_list = df[is_collection_member]
    
    if b_include_pages == True:
        is_part_of_book =  df['isPageOf'] == collection
        member_list_pages = df[is_part_of_book]
        member_list = member_list.append(member_list_pages,ignore_index=True)
        
    ## Loop through each member, if it is a collection model, then call this again!
    for index, row in member_list.iterrows():
        pid = row["PID"]
        cmodel = row["cmodel"]
        is_member_of_collection = row["isMemberOfCollection"]
        if cmodel == "info:fedora/islandora:collectionCModel" or cmodel == "info:fedora/islandora:bookCModel":
            df_collection_members = get_members(df, df_collection_members, "info:fedora/"+pid, b_include_pages)
        
    df_collection_members = df_collection_members.append(member_list,ignore_index=True)
    return df_collection_members

In [86]:
#https://riptutorial.com/pandas/example/25462/split--reshape--csv-strings-in-columns-into-multiple-rows--having-one-element-per-row
# chain methods
# 1. Sets the entire dataframe as index, except the list or specified column
# 2. split the column to multiple columns
# 3. transpose or stack it
# 4. reset index to original !
# 5. rename the 0 column
# 6. 
def reshape_column_row(df, column_name, new_name):
    df_reshape = \
    (df.set_index(df.columns.drop(column_name,1).tolist())
       .loc[:, column_name].str.split(',', expand=True)
       .stack()
       .reset_index()
       .rename(columns={0:column_name})
       .loc[:, df.columns]
    )
    df_reshape.rename(columns={column_name:new_name}, inplace=True)
    return df_reshape

In [87]:
df = pd.read_csv("data/repo_data.csv") 
df.checksum_list = df.checksum_list.fillna('not_set')
df['checksum_count'] = df.apply(lambda x: count_checksums(x['checksum_list']), axis=1)
df.head(5)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream_list,datastreams_count,OBJ_Exists,obj_checksum,checksum_list,Premis_Check,Last_Modified_By,obj_format,mimetypes_list,title,size_list,fgs_lastModifiedDate_dt,datastream_versions,checksum_count
0,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,"not_set, 94e8f30a05c70096e53be5b1dcb6066769dbb...",Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",8
1,islandora:10,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,"not_set, d1c419dba0ca9e133dcdd478ad654b728e1f7...",Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",back page,"not_set, 718, 1258, 452, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",8
2,example:3,NaN,info:fedora/example:1,info:fedora/islandora:pageCModel,"AUDIT, RELS-EXT, PDF, OBJ, DC, TN, JP2, TECHMD...",10,OBJ,2aff5cfebf859d7933ee76bba594b1ede8e30f9e,"not_set, 3092d6667f455f236e48a737b1cbd12cbe17a...",Validated,admin,TIFF EXIF,"text/xml, application/rdf+xml, application/pdf...",sample.pdf-2,"not_set, 822, 6636, 280983, 374, 188825, 13813...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1, 1, 2",10
3,example:2,NaN,info:fedora/example:1,info:fedora/islandora:pageCModel,"AUDIT, RELS-EXT, PDF, OBJ, DC, TN, JP2, TECHMD...",10,OBJ,cd3b241ed4869776bb90387b51766e04b7a62151,"not_set, 0b34e4584546dc42ef0f0209ca1105ce87cc7...",Validated,admin,TIFF EXIF,"text/xml, application/rdf+xml, application/pdf...",sample.pdf-1,"not_set, 822, 6185, 298854, 374, 189257, 14929...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1, 1, 2",10
4,islandora:8,info:fedora/utsc:collection2,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,f4e806111bdb1c39c1e281e342afd59fed40d198,"not_set, 8a77ac9ee1a9edca8183cd33525a5759bc2ff...",Validated,admin,JPEG EXIF,"text/xml, application/rdf+xml, application/xml...",aaaa,"not_set, 546, 1253, 446, 3773963, 11028, 29019...","Tuesday, March 26, 2019 - 13:15","1, 1, 1, 1, 1, 1, 1, 1",8


## Namespace Info

In [88]:
# Objects by namespace
df['namespace'] = df.apply(lambda x: get_namespace(x['PID']), axis=1)
df_namespace = df.groupby('namespace')["PID"].count().reset_index(name="count")

In [89]:
df_namespace.to_csv("reports/objets_by_namespace.csv")

## Collections info

In [90]:
collection = "info:fedora/islandora:sp_basic_image_collection"
collection_info = collection.split(":")
collection_name = collection_info[2]
collection_name

'sp_basic_image_collection'

In [91]:
# List of conceptual objects (includes objects that are part of compound model)
# For compound objects, Hide child objects in Solr results  should be turned off
df_collection_members = pd.DataFrame()
df_collection_members = get_members(df, df_collection_members, collection, False)
df_collection_members.head(5)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream_list,datastreams_count,OBJ_Exists,obj_checksum,checksum_list,Premis_Check,Last_Modified_By,obj_format,mimetypes_list,title,size_list,fgs_lastModifiedDate_dt,datastream_versions,checksum_count,namespace
0,islandora:8,info:fedora/utsc:collection2,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,f4e806111bdb1c39c1e281e342afd59fed40d198,"not_set, 8a77ac9ee1a9edca8183cd33525a5759bc2ff...",Validated,admin,JPEG EXIF,"text/xml, application/rdf+xml, application/xml...",aaaa,"not_set, 546, 1253, 446, 3773963, 11028, 29019...","Tuesday, March 26, 2019 - 13:15","1, 1, 1, 1, 1, 1, 1, 1",8,islandora
1,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,"not_set, 94e8f30a05c70096e53be5b1dcb6066769dbb...",Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",8,islandora
2,islandora:10,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,"not_set, d1c419dba0ca9e133dcdd478ad654b728e1f7...",Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",back page,"not_set, 718, 1258, 452, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",8,islandora
3,islandora:6,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,4784c50df21ca2fa8f2a3219adcaf9dbbffe7ae8,"not_set, a4eeb4470e100b73ee58662ba364fac89e9bb...",Validated,admin,JPEG EXIF,"text/xml, application/rdf+xml, application/xml...",inside object 2,"not_set, 541, 1264, 457, 9041285, 10422, 70911...","Tuesday, March 26, 2019 - 13:15","1, 1, 1, 1, 1, 1, 1, 1",8,islandora
4,islandora:5,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,4e424da322f285aa179caec35397d161ab76c075,"not_set, 6c11a823b8451a7f9b1ffc417d981ea2a2f0d...",Validated,admin,JPEG EXIF,"text/xml, application/rdf+xml, application/xml...",inside object 1,"not_set, 541, 1264, 457, 9240579, 10422, 71002...","Tuesday, March 26, 2019 - 13:15","1, 1, 1, 1, 1, 1, 1, 1",8,islandora


In [92]:
df_collection_members.to_csv("reports/objects_by_collections_" + collection_name + ".csv")

In [93]:
# List of objects (including pages)
df_collection_members = pd.DataFrame()
df_collection_members = get_members(df, df_collection_members, collection, True)
df_collection_members.head(5)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream_list,datastreams_count,OBJ_Exists,obj_checksum,checksum_list,Premis_Check,Last_Modified_By,obj_format,mimetypes_list,title,size_list,fgs_lastModifiedDate_dt,datastream_versions,checksum_count,namespace
0,example:3,NaN,info:fedora/example:1,info:fedora/islandora:pageCModel,"AUDIT, RELS-EXT, PDF, OBJ, DC, TN, JP2, TECHMD...",10,OBJ,2aff5cfebf859d7933ee76bba594b1ede8e30f9e,"not_set, 3092d6667f455f236e48a737b1cbd12cbe17a...",Validated,admin,TIFF EXIF,"text/xml, application/rdf+xml, application/pdf...",sample.pdf-2,"not_set, 822, 6636, 280983, 374, 188825, 13813...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1, 1, 2",10,example
1,example:2,NaN,info:fedora/example:1,info:fedora/islandora:pageCModel,"AUDIT, RELS-EXT, PDF, OBJ, DC, TN, JP2, TECHMD...",10,OBJ,cd3b241ed4869776bb90387b51766e04b7a62151,"not_set, 0b34e4584546dc42ef0f0209ca1105ce87cc7...",Validated,admin,TIFF EXIF,"text/xml, application/rdf+xml, application/pdf...",sample.pdf-1,"not_set, 822, 6185, 298854, 374, 189257, 14929...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1, 1, 2",10,example
2,islandora:8,info:fedora/utsc:collection2,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,f4e806111bdb1c39c1e281e342afd59fed40d198,"not_set, 8a77ac9ee1a9edca8183cd33525a5759bc2ff...",Validated,admin,JPEG EXIF,"text/xml, application/rdf+xml, application/xml...",aaaa,"not_set, 546, 1253, 446, 3773963, 11028, 29019...","Tuesday, March 26, 2019 - 13:15","1, 1, 1, 1, 1, 1, 1, 1",8,islandora
3,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,"not_set, 94e8f30a05c70096e53be5b1dcb6066769dbb...",Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",8,islandora
4,islandora:10,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,"AUDIT, RELS-EXT, MODS, DC, OBJ, TECHMD, TN, ME...",8,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,"not_set, d1c419dba0ca9e133dcdd478ad654b728e1f7...",Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",back page,"not_set, 718, 1258, 452, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",8,islandora


In [94]:
df_collection_members.to_csv("reports/objects_by_collections_" + collection_name + "_including_pages.csv")

## Datastream Info

In [95]:
# Get datastreams 
df_datastreams_report = reshape_column_row(df, "datastream_list", "datastream")
del df_datastreams_report["datastreams_count"]
df_datastreams_report.head(2)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream,OBJ_Exists,obj_checksum,checksum_list,Premis_Check,Last_Modified_By,obj_format,mimetypes_list,title,size_list,fgs_lastModifiedDate_dt,datastream_versions,checksum_count,namespace
0,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,AUDIT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,"not_set, 94e8f30a05c70096e53be5b1dcb6066769dbb...",Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",8,islandora
1,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,RELS-EXT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,"not_set, 94e8f30a05c70096e53be5b1dcb6066769dbb...",Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",8,islandora


In [96]:
df_checksum = reshape_column_row(df, "checksum_list", "checksum")
df_datastreams_report["checksum"] = df_checksum["checksum"]
del df_datastreams_report["checksum_list"]
del df_datastreams_report["checksum_count"]
df_datastreams_report.head(2)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream,OBJ_Exists,obj_checksum,Premis_Check,Last_Modified_By,obj_format,mimetypes_list,title,size_list,fgs_lastModifiedDate_dt,datastream_versions,namespace,checksum
0,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,AUDIT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",islandora,not_set
1,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,RELS-EXT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF","text/xml, application/rdf+xml, application/xml...",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",islandora,94e8f30a05c70096e53be5b1dcb6066769dbb6fd


In [97]:
df_mimetype = reshape_column_row(df, "mimetypes_list", "mimetype")
df_datastreams_report["mimetype"] = df_mimetype["mimetype"]
del df_datastreams_report["mimetypes_list"]
df_datastreams_report.head(2)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream,OBJ_Exists,obj_checksum,Premis_Check,Last_Modified_By,obj_format,title,size_list,fgs_lastModifiedDate_dt,datastream_versions,namespace,checksum,mimetype
0,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,AUDIT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",islandora,not_set,text/xml
1,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,RELS-EXT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"not_set, 718, 1260, 454, 1878576, 3439, 29414,...","Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",islandora,94e8f30a05c70096e53be5b1dcb6066769dbb6fd,application/rdf+xml


In [98]:
df_size = reshape_column_row(df, "size_list", "size")
df_datastreams_report["size"] = df_size["size"]
del df_datastreams_report["size_list"]
df_datastreams_report.head(2)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream,OBJ_Exists,obj_checksum,Premis_Check,Last_Modified_By,obj_format,title,fgs_lastModifiedDate_dt,datastream_versions,namespace,checksum,mimetype,size
0,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,AUDIT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",islandora,not_set,text/xml,not_set
1,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,RELS-EXT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15","1, 2, 1, 1, 1, 1, 1, 1",islandora,94e8f30a05c70096e53be5b1dcb6066769dbb6fd,application/rdf+xml,718


In [99]:
df_versions = reshape_column_row(df, "datastream_versions", "num_versions")
df_datastreams_report["num_versions"] = df_versions["num_versions"]
del df_datastreams_report["datastream_versions"]
df_datastreams_report.head(2)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream,OBJ_Exists,obj_checksum,Premis_Check,Last_Modified_By,obj_format,title,fgs_lastModifiedDate_dt,namespace,checksum,mimetype,size,num_versions
0,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,AUDIT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15",islandora,not_set,text/xml,not_set,1
1,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,RELS-EXT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15",islandora,94e8f30a05c70096e53be5b1dcb6066769dbb6fd,application/rdf+xml,718,2


In [100]:
df_datastreams_report.to_csv("reports/all_datastreams.csv")

In [101]:
# Size report

df_datastreams_report['size'].replace("not_set", '0',inplace=True)
df_datastreams_report = df_datastreams_report.astype({"size": int})
df_datastreams_report = df_datastreams_report.astype({"num_versions": int})
df_datastreams_report['total_size'] = df_datastreams_report['size'] * df_datastreams_report['num_versions']
df_datastreams_report.head(5)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream,OBJ_Exists,obj_checksum,Premis_Check,Last_Modified_By,obj_format,title,fgs_lastModifiedDate_dt,namespace,checksum,mimetype,size,num_versions,total_size
0,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,AUDIT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15",islandora,not_set,text/xml,0,1,0
1,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,RELS-EXT,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15",islandora,94e8f30a05c70096e53be5b1dcb6066769dbb6fd,application/rdf+xml,718,2,1436
2,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,MODS,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15",islandora,bafe21cbde340c0764c8313038623d10290386c6,application/xml,1260,1,1260
3,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,DC,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15",islandora,4ca48ca4c7ca3d9964b33485baa1e9c5510e31cf,text/xml,454,1,454
4,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,OBJ,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15",islandora,22ace46f16f064d6c24fe8027030907165ee9eca,image/jpeg,1878576,1,1878576


In [107]:
df_size = df_datastreams_report.groupby('datastream')["total_size"].sum().reset_index(name="sum")
df_size = df_size.sort_values(by=['sum'], ascending=False)
df_size.to_csv("reports/storage_size_by_datastream.csv")
df_size

,datastream,sum
7,OBJ,35819318
12,TN,991020
4,JPG,495236
5,MEDIUM_SIZE,470358
3,JP2,287429
11,TECHMD,77204
6,MODS,29310
9,RELS-EXT,18487
2,DC,16191
8,PDF,15849


In [103]:
df_datastreams_report['datastream'] = df_datastreams_report['datastream'].str.strip()
is_obj =  df_datastreams_report['datastream'] == "OBJ"
objs = df_datastreams_report[is_obj]
objs.head(5)

,PID,isMemberOfCollection,isPageOf,cmodel,datastream,OBJ_Exists,obj_checksum,Premis_Check,Last_Modified_By,obj_format,title,fgs_lastModifiedDate_dt,namespace,checksum,mimetype,size,num_versions,total_size
4,islandora:11,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,OBJ,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",second page,"Tuesday, March 26, 2019 - 13:15",islandora,22ace46f16f064d6c24fe8027030907165ee9eca,image/jpeg,1878576,1,1878576
12,islandora:10,info:fedora/example:nat,NaN,info:fedora/islandora:sp_basic_image,OBJ,OBJ,22ace46f16f064d6c24fe8027030907165ee9eca,Validated,admin,"JPEG File Interchange Format, JPEG EXIF",back page,"Tuesday, March 26, 2019 - 13:15",islandora,22ace46f16f064d6c24fe8027030907165ee9eca,image/jpeg,1878576,1,1878576
19,example:3,NaN,info:fedora/example:1,info:fedora/islandora:pageCModel,OBJ,OBJ,2aff5cfebf859d7933ee76bba594b1ede8e30f9e,Validated,admin,TIFF EXIF,sample.pdf-2,"Tuesday, March 26, 2019 - 13:15",example,2aff5cfebf859d7933ee76bba594b1ede8e30f9e,image/tiff,280983,1,280983
29,example:2,NaN,info:fedora/example:1,info:fedora/islandora:pageCModel,OBJ,OBJ,cd3b241ed4869776bb90387b51766e04b7a62151,Validated,admin,TIFF EXIF,sample.pdf-1,"Tuesday, March 26, 2019 - 13:15",example,cd3b241ed4869776bb90387b51766e04b7a62151,image/tiff,298854,1,298854
40,islandora:8,info:fedora/utsc:collection2,NaN,info:fedora/islandora:sp_basic_image,OBJ,OBJ,f4e806111bdb1c39c1e281e342afd59fed40d198,Validated,admin,JPEG EXIF,aaaa,"Tuesday, March 26, 2019 - 13:15",islandora,f4e806111bdb1c39c1e281e342afd59fed40d198,image/jpeg,3773963,1,3773963


In [104]:
# Get total size of total OBJ objects (latest version)
objs = objs.astype({"size": int})
objs["size"].sum()

32045355

In [105]:
# Get file formats
df_file_formats = df.groupby('obj_format')["PID"].count().reset_index(name="count")
df_file_formats

,obj_format,count
0,JPEG EXIF,4
1,"JPEG File Interchange Format, JPEG EXIF",3
2,TIFF EXIF,2


In [106]:
df_file_formats.to_csv("reports/objs_by_fileformats.csv")